## CASE 3/CASE 6

### Summary/Introduction blurb

### Data Processing 

We received the data in an "R data format" file. Since we opted to do this analysis in Python, the Python package "pyreadr" was used to read the data into a Python Pandas dataframe; we then converted the dataframe to a comma separated file (CSV) to store on GitHub for ease of use. Each execution of this Jupyter notebook reads the data from the CSV file on GitHub. 

https://github.com/jjschueder/7333QTW/blob/master/Case%20Study%203/Py%20read%20R.JPG -- DO WE NEED THIS LINK?

In [1]:
#import all packages needed
import pandas as pd
import pyreadr
from pandas_profiling import ProfileReport

ModuleNotFoundError: No module named 'pyreadr'

In [ ]:
# import the dataset - ONLY DONE ONCE, included for reference
# result = pyreadr.read_r(r'C:\Users\jjsch\Downloads\Week_5_Materials_2\data.Rda')

In [ ]:
# convert to a dataframe - ONLY DONE ONCE, for reference
# df = result["emailDFrp"]

In [ ]:
# export dataframe to CSV - ONLY DONE ONCE, for reference
# df.to_csv(r'C:\Users\jjsch\Downloads\Week_5_Materials_2\data.csv')

In [2]:
# view data 
df = pd.read_csv(r'https://raw.githubusercontent.com/jjschueder/7333QTW/master/Case%20Study%203/data.csv')
df.head()

,Unnamed: 0,isSpam,isRe,underscore,priority,isInReplyTo,sortedRec,subPunc,multipartText,hasImages,...,subQuesCt,numAtt,numRec,perCaps,hour,perHTML,subBlanks,forwards,avgWordLen,numDlr
0,0,0,T,F,F,T,T,F,F,F,...,0.0,0.0,2.0,4.451039,11.0,0.0,12.500000,0.000000,4.376623,3
1,1,0,F,F,F,F,T,F,F,F,...,0.0,0.0,1.0,7.491289,11.0,0.0,8.000000,0.000000,4.555556,0
2,2,0,F,F,F,F,T,F,F,F,...,0.0,0.0,1.0,7.436096,12.0,0.0,8.000000,0.000000,4.817164,0
3,3,0,F,F,F,F,T,F,F,F,...,0.0,0.0,0.0,5.090909,13.0,0.0,18.918919,3.125000,4.714286,0
4,4,0,T,F,F,F,T,F,F,F,...,0.0,0.0,1.0,6.116643,13.0,0.0,15.217391,6.451613,4.234940,0


Our initial review of the data shows many columns are formated as a "logical" data type of TRUE or FALSE. Machine learning classificaton algorithms need numeric variables. Therefore, we converted the TRUE/FALSE variables to a numeric representation: TRUE == 1; FALSE == 0. The remaining variables are numeric as they represent percentages, counts, and hours of the day.

In [3]:
# Transform logical data to numeric data
# df['isSpam'] = df['isSpam'].map({'T': 1, 'F': 0})
df['isRe'] = df['isRe'].map({'T': 1, 'F': 0})             
df['underscore'] = df['underscore'].map({'T': 1, 'F': 0})       
df['priority'] = df['priority'].map({'T': 1, 'F': 0})         
df['isInReplyTo'] = df['isInReplyTo'].map({'T': 1, 'F': 0})      
df['sortedRec'] = df['sortedRec'].map({'T': 1, 'F': 0})        
df['subPunc'] = df['subPunc'].map({'T': 1, 'F': 0})          
df['multipartText'] = df['multipartText'].map({'T': 1, 'F': 0})    
df['hasImages'] = df['hasImages'].map({'T': 1, 'F': 0})        
df['isPGPsigned'] = df['isPGPsigned'].map({'T': 1, 'F': 0})       
df['subSpamWords'] = df['subSpamWords'].map({'T': 1, 'F': 0})      
df['noHost'] = df['noHost'].map({'T': 1, 'F': 0})            
df['numEnd'] = df['numEnd'].map({'T': 1, 'F': 0})            
df['isYelling'] = df['isYelling'].map({'T': 1, 'F': 0})         
df['isOrigMsg'] = df['isOrigMsg'].map({'T': 1, 'F': 0})         
df['isDear'] = df['isDear'].map({'T': 1, 'F': 0})            
df['isWrote'] = df['isWrote'].map({'T': 1, 'F': 0})    

### Exploratory Data Analysis 

In [4]:
type(df)

pandas.core.frame.DataFrame

After confirming proper the proper type of data frame, we conducted exploratory data analysis to better understand the dataset. Our first step was to understand the variables and their associated meaning in relation to an email message. The data dictionary below provides a summary. Note, the target variable to predict - IsSpam - is included. There is a record id - Unnamed: 0 - which is not necessary for our analysi,s but we retained it for reference as needed.

#### Data Dictionary

In [5]:
df.columns

Index(['Unnamed: 0', 'isSpam', 'isRe', 'underscore', 'priority', 'isInReplyTo',
       'sortedRec', 'subPunc', 'multipartText', 'hasImages', 'isPGPsigned',
       'subSpamWords', 'noHost', 'numEnd', 'isYelling', 'isOrigMsg', 'isDear',
       'isWrote', 'numLines', 'bodyCharCt', 'subExcCt', 'subQuesCt', 'numAtt',
       'numRec', 'perCaps', 'hour', 'perHTML', 'subBlanks', 'forwards',
       'avgWordLen', 'numDlr'],
      dtype='object')

| VARIABLE | DEFINITION |
|-|-|
| isSpam | TRUE if email classified as spam |
| isRe | TRUE if Re: appears at the start of subject line |
| numLines | number of lines in body of message |
| bodyCharCt | number of characters in the body of message |
| underscore | TRUE if email address in From field of header contains underscore |
| subExcCt | number of exclamation marks in subject |
| subQuesCt | number of question marks in subject |
| numAtt | number of attachments in message |
| priority | TRUE if Priority key is present in header |
| numRec | number of recipients of message, including CCs |
| perCaps | percentage of capitals among all letters in message |
| isInReplyTo | TRUE if the In-Reply-To key is present in header |
| sortedRec | TRUE if recipients' email addresses are sorted |
| subPunc | TRUE if words in subject have punctuation or numbers embedded (i.e. w!se) |
| hour | hour of the day in the Date field |
| multipartText | TRUE if MIME type is multipart/text |
| hasImages | TRUE if message contains images |
| isPGPsigned | TRUE if message contains a PGP (encryption) signature |
| perMTML | percentage of characters in HTML tags in message compared to all characters |
| subSpamWords | TRUE if subject contains one of the words in spam word vector |
| subBlanks | percentage of blanks in subject |
| noHost | TRUE if there is no hostname in Message-Id key in header |
| numEnd | TRUE if sender's email address (before @) ends in number |
| isYelling | TRUE if subject is in all capital letters |
| forwards | number of forward symbols in a line of the body (i.e. >>> xxx contains 3) |
| isOrigMsg | TRUE if message body contains phrase "original message" |
| isDear | TRUE if message body contains word "dear" |
| isWrote | TRUE if message contains phrase "wrote:" |
| avgWordLen | average length of words in message |
| numDlr | number of dollar signs in message |

#### Data profiling

Next, we dug deeper into the exploratory analysis to understand descriptive statistics, data distrutions, and corrleation between our variables. Using standard Python command and a Python package called Pandas Profiling, we found some notable characters about select variables.

Missing data: 
* subSpam (7 missing), noHost (1 missing), isYelling (7 missing), subExcCt (20 missing), subQuesCt (20 missing), numRec (20 missing), and subBlanks (20 missing). 
* Missing values and imputation strategy outlined below.

Outliers / Odd data:
* Outliers correlated with Spam: numRec with 311 receipents; 100% capital letters in email (>80% was spam threshold for perCaps); 100% HTML characters (>97% was spam threshold for perHTML); 86% blanks in subject (>25% was spam threshold for subBlanks), and average word length of 26 is a spam outlier.
* Outliers not necessarily correlated with Spam: count of question marks in subject (8 and 12 outliers were not spam); greatest number of attachments at 18 (not spam); number of forwards at 99 (not spam); and large number of dollar signs at 1977 (not spam).

WHAT OTHER INTERESTING THINGS TO NOTE FROM PROFILE
* Distributions
* Other thoughts?


In [6]:
df.describe()

,Unnamed: 0,isSpam,isRe,underscore,priority,isInReplyTo,sortedRec,subPunc,multipartText,hasImages,...,subQuesCt,numAtt,numRec,perCaps,hour,perHTML,subBlanks,forwards,avgWordLen,numDlr
count,9348.000000,9348.000000,9348.000000,9348.000000,9348.000000,9348.000000,9348.000000,9348.000000,9348.000000,9348.000000,...,9328.000000,9348.000000,9066.000000,9348.000000,9348.000000,9348.000000,9328.000000,9348.000000,9348.000000,9348.000000
mean,4673.500000,0.256418,0.321459,0.013479,0.005777,0.298674,0.898588,0.028134,0.035088,0.002353,...,0.137757,0.065789,1.929407,8.850371,12.210847,6.517082,13.866939,10.445086,4.487222,1.781558
std,2698.679492,0.436679,0.467062,0.115319,0.075788,0.457701,0.301890,0.165365,0.184011,0.048458,...,0.507685,0.324879,5.242396,9.583415,6.623932,19.135266,7.431938,18.263576,0.568582,30.380455
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.363072,0.000000
25%,2336.750000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,4.255319,8.000000,0.000000,10.526316,0.000000,4.208257,0.000000
50%,4673.500000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,6.055473,13.000000,0.000000,13.253012,0.000000,4.454545,0.000000
75%,7010.250000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,9.059399,18.000000,0.000000,15.686275,15.384615,4.728507,0.000000
max,9347.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,12.000000,18.000000,311.000000,100.000000,23.000000,100.000000,86.419753,99.058270,26.000000,1977.000000


In [7]:
profile = ProfileReport(df, title='Pandas Profiling Report', explorative=True)

NameError: name 'ProfileReport' is not defined

In [8]:
profile.to_notebook_iframe()

NameError: name 'profile' is not defined

#### Missing Values and Imputation Strategy
As noted above, we addressed missing values in our dataset to improve our results. We opted to drop the 20 rows missing in the “subExcCt”, “subQuesCt”, and “subBlanks” columns. The missing data associated with these three columns was absent in the same rows; therefore, only 20 records were removed in addressing those columns. We felt this was an appropriate step given it represented only 0.2% of the data. Dropping these records also removed the seven missing values of “isYelling” and “subSpamWords.”

The number of recipients (numRec) has the most missing values at 282. We decided to impute the missing values with the mean number of recipients. This impution was done after we split the data into training and testing datasets. The mean was calculated from the training dataset only. We felt this would most closely mimic a production environment and limit data leak between training and test sets. The remaining missing data point was one value for “noHost"; we imputed this with the most frequently occurring value (mode) of False (0). **{IS THIS WHAT WE DID!??}{ OR WE CAN ASSIGN THE VALUE SINCE WE KNOW THE MISSING ONE IS SPAM ... I think all the SPAMs have a noHost of TRUE}

In [9]:
# Missing Data Summary
countOfNan = pd.Series(df.isnull().sum()) 
DataType = pd.Series(df.dtypes) 
  
frame = { 'datatype': DataType, 'count of Nan': countOfNan } 
result = pd.DataFrame(frame) 
print(result)

# remove NaNs
dfNoNa=df[~df['subQuesCt'].isnull()]

              datatype  count of Nan
Unnamed: 0       int64             0
isSpam           int64             0
isRe             int64             0
underscore       int64             0
priority         int64             0
isInReplyTo      int64             0
sortedRec        int64             0
subPunc          int64             0
multipartText    int64             0
hasImages        int64             0
isPGPsigned      int64             0
subSpamWords   float64             7
noHost         float64             1
numEnd           int64             0
isYelling      float64             7
isOrigMsg        int64             0
isDear           int64             0
isWrote          int64             0
numLines         int64             0
bodyCharCt       int64             0
subExcCt       float64            20
subQuesCt      float64            20
numAtt         float64             0
numRec         float64           282
perCaps        float64             0
hour           float64             0
p

### Explanation of Data Split, equivalent to R split()

In the Python sklearn package, "Statified Shuffle Split" provides train/test indices to split data in train/test sets.
This cross-validation object is a merge of StratifiedKFold and ShuffleSplit, which returns stratified randomized folds. The folds are made by preserving the percentage of samples for each class. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html  

The settings we use for training are 1 fold using eighty percent of the data for training. Twenty percent of the data will be used to test the initial model. 

** DID WE USE STRATIFIED SHUFFLE SPLIT OR JUST SHUFFLE SPLIT ?? It looks like we commented out Strat Shuffle Split. Why did we decide to do that? Why was Shuffle Split better than Strat Shuffle Split??

In [10]:
# now divide the data into test and train using scikit learn built-ins
from sklearn.model_selection import StratifiedShuffleSplit 
from sklearn.model_selection import ShuffleSplit
cvx = ShuffleSplit(n_splits=10, test_size=0.20, random_state=101)
#cv = StratifiedShuffleSplit(n_splits=10,train_size=0.8)
print (cvx)

ShuffleSplit(n_splits=10, random_state=101, test_size=0.2, train_size=None)


In [11]:
features = ['isRe', 'underscore', 'priority', 'isInReplyTo',
       'sortedRec', 'subPunc', 'multipartText', 'hasImages', 'isPGPsigned',
       'subSpamWords', 'noHost', 'numEnd', 'isYelling', 'isOrigMsg', 'isDear',
       'isWrote', 'numLines', 'bodyCharCt', 'subExcCt', 'subQuesCt', 'numAtt',
       'numRec', 'perCaps', 'hour', 'perHTML', 'subBlanks', 'forwards',
       'avgWordLen', 'numDlr']

X = dfNoNa[features].copy()

#scaler = StandardScaler()
#scaler.fit(X2)

#This makes our model's coefficients take on the same scale for accurate feature importance analysis
#Notice we scaled the data before the cross validation
#X = scaler.transform(X2)

Y= dfNoNa[['isSpam']].copy()
y = Y.values

### Q1: BUILD/EVALUATE TREE-BASED MODEL FOR PREDICTING "SPAM"

Baseline, complex model
explain what we did, why we did this, summarize the findings.

First the data is split into random train and test subsets, then a grid search mechanism is utilized to train models with various parmeters, score those models in terms of accuracy and return which model will provide the best accuracy.

** RAN BASELINE MODEL -- with accuracy of 96%. can we get other stats about the baseline tree: how many leaves/levels, what were the results of the other metrics. 

** We could define the metrics to evaluate models here (before we run baseline) to be able to show the full change in results between base and our defined model.


#### SUMMARY BASELINE MODEL
Accuracy 96.67% <br>
WHAT ELSE -- how many layers? <br>
What key variables? All used? <br>

***WHAT IF WE USED GRID SEARCH AS OUR FIRST STEP TO UNDERSTAND WHAT PERFORMS BEST. USE THAT AS THE BASELINE, and say, great model but too complex. So we explored all the options output by grid search and created THIS model.

If we go this route, we only have 2 models to describe. 

In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, TimeSeriesSplit, StratifiedShuffleSplit
from sklearn import metrics as mt
X1_train, X1_test, y1_train, y1_test = train_test_split(X, y, test_size = 0.2, random_state = 101)

#impute missing values for noHost and numRec using the mean of the training set
X1_train.loc[X1_train['numRec'].isnull(),'numRec']=X1_train.numRec.mean(skipna = True)
X1_test.loc[X1_test['numRec'].isnull(),'numRec']=X1_train.numRec.mean(skipna = True)
X1_train.loc[X1_train['noHost'].isnull(),'noHost']=X1_train.noHost.mean(skipna = True)
X1_test.loc[X1_test['noHost'].isnull(),'noHost']=X1_train.noHost.mean(skipna = True)

#run base line decision tree classifier 
dt_clf = DecisionTreeClassifier()
%time dt_clf.fit(X1_train,y1_train)
yhat = dt_clf.predict(X1_test)
print ('accuracy:', mt.accuracy_score(y1_test,yhat))
print (dt_clf)

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Wall time: 49.9 ms
accuracy: 0.9678456591639871
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')


In [14]:
#grid search
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


min_samples_leaf_range = [2,3,4,5,6,7,8,9,10,11,15,20,30,40,50,75,100,150]
# Create a dictionary of all the parameter options 
# Note has you can access the parameters of steps of a pipeline by using '__’



parameters ={'classify__criterion':['gini','entropy'],
             'classify__max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150],
             'classify__min_samples_leaf': min_samples_leaf_range,
             'classify__max_features': ['auto','auto', 'log2']}


estimator = Pipeline([("imputer", SimpleImputer(missing_values=np.nan, strategy='mean')),
                      ("classify",DecisionTreeClassifier())])

The following is doing a grid search, the cv=cvx is telling the grid search to split the training and testing data 10 times. 

In [15]:
dt_clf = GridSearchCV(estimator, param_grid =parameters, cv=cvx)


# train the decision tree algorithm
%time dt_clf.fit(X,y)
yhat = dt_clf.best_estimator_.predict(X1_test)
print ('accuracy:', mt.accuracy_score(y1_test,yhat))
print (dt_clf.best_params_)

Wall time: 9min 8s
accuracy: 0.9833869239013934
{'classify__criterion': 'gini', 'classify__max_depth': 70, 'classify__max_features': 'auto', 'classify__min_samples_leaf': 2}


In [16]:
#commented as it was not doing the cv split. replaced with cell immediately above.
#dt_clf = GridSearchCV(DecisionTreeClassifier(), param_grid =parameters)


# train the decision tree algorithm
#%time dt_clf.fit(X1_train,y1_train)
#yhat = dt_clf.best_estimator_.predict(X1_test)
#print ('accuracy:', mt.accuracy_score(y1_test,yhat))
#print (dt_clf.best_params_)
#Wall time: 1min 2s
#accuracy: 0.9463792150359315
#{'criterion': 'gini', 'max_depth': 120, 'max_features': 'auto', 'min_samples_leaf': 2}

### Q3: EXPLAIN PARAMETERS INVOLVED IN "TUNING" MODEL

Explain our decisions, how that impacted from base model
Parameter definitions (if we need them)

| Parameter                | Definition                                                                                                                                                                                  |
|--------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Criterion                | function to measure the quality of a split; supported criteria are "gini" for Gini impurity and "entropy" for information gain                                                              |
| Splitter                 | strategy used to choose best split at each node; supported strategies are "best" to choose best split and "random  to choose best random split                                              |
| Max_depth                | maximum depth of tree; if None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples                                           |
| Min_samples_split        | minimum number of samples required to split an internal node                                                                                                                                |
| Min_samples_leaf         | minimum number of samples required at a leaf node; split point at any depth will only be considered if it leaves at least min_samples_leaf training samples in each left and right branches |
| Min_weight_fraction_leaf | minimum weighted fraction of sum total of weights (for all input samples) required to be at a leaf node; samples have equal weight when sample_weight is not provided                       |
| Max_features             | number of features to consider when looking for best split                                                                                                                                  |
| Random_state             | controls randomness of estimator                                                                                                                                                            |
| Max_leaf_nodes           | grow a tree with max_leaf_nodes in best-first fashion; best nodes defined as relative reduction in impurity                                                                                 |
| Min_impurity_decrease    | node will be split if this split induces a decrease of impurity greater than/equal to this value                                                                                            |
| Class_weight             | weights associated with classes                                                                                                                                                             |
| Ccp_alpha                | complexity parameter used for Minimal Cost-Complexity Pruning                                                                                                                               |

In [17]:
estimatorsdf = pd.DataFrame(dt_clf.cv_results_)

C:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are acc

In [18]:
estimatorsdf

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classify__criterion,param_classify__max_depth,param_classify__max_features,param_classify__min_samples_leaf,params,split0_test_score,...,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,split8_train_score,split9_train_score,mean_train_score,std_train_score
0,0.017106,0.006873,0.001801,4.001719e-04,gini,4,auto,2,"{'classify__criterion': 'gini', 'classify__max...",0.874598,...,0.859019,0.885687,0.843474,0.853122,0.847762,0.852452,0.860627,0.857947,0.852171,0.023556
1,0.014724,0.000774,0.001795,3.933949e-04,gini,4,auto,3,"{'classify__criterion': 'gini', 'classify__max...",0.878885,...,0.844814,0.861029,0.854463,0.854061,0.850978,0.871080,0.860225,0.797641,0.851233,0.019532
2,0.015882,0.000814,0.001701,4.427542e-04,gini,4,auto,4,"{'classify__criterion': 'gini', 'classify__max...",0.881029,...,0.806352,0.838381,0.831412,0.871750,0.821630,0.877111,0.848298,0.840391,0.842227,0.024725
3,0.014483,0.001111,0.001700,4.440328e-04,gini,4,auto,5,"{'classify__criterion': 'gini', 'classify__max...",0.881565,...,0.846824,0.875905,0.829402,0.855669,0.864380,0.820557,0.840257,0.859153,0.847682,0.019071
4,0.017057,0.001445,0.002007,4.377215e-04,gini,4,auto,6,"{'classify__criterion': 'gini', 'classify__max...",0.824759,...,0.869070,0.878719,0.863977,0.840525,0.832485,0.860627,0.871750,0.882605,0.858081,0.020361
5,0.016569,0.001009,0.001907,2.931534e-04,gini,4,auto,7,"{'classify__criterion': 'gini', 'classify__max...",0.858521,...,0.888904,0.788931,0.857143,0.871348,0.867998,0.846288,0.851648,0.885687,0.857679,0.026431
6,0.015668,0.000795,0.001897,5.383609e-04,gini,4,auto,8,"{'classify__criterion': 'gini', 'classify__max...",0.858521,...,0.838649,0.854463,0.852586,0.839453,0.870812,0.866524,0.824578,0.859689,0.851126,0.013270
7,0.015854,0.001043,0.001804,3.951937e-04,gini,4,auto,9,"{'classify__criterion': 'gini', 'classify__max...",0.849946,...,0.879657,0.894130,0.830206,0.854329,0.853659,0.869606,0.835835,0.813187,0.852117,0.022857
8,0.014967,0.000777,0.001496,4.982250e-04,gini,4,auto,10,"{'classify__criterion': 'gini', 'classify__max...",0.858521,...,0.884749,0.839989,0.856473,0.869070,0.856339,0.838247,0.811043,0.852586,0.849558,0.018935
9,0.022661,0.007073,0.003296,2.037102e-03,gini,4,auto,11,"{'classify__criterion': 'gini', 'classify__max...",0.851018,...,0.820960,0.870410,0.863709,0.860627,0.868802,0.857277,0.868936,0.824444,0.850750,0.019105


Refit with the best params to use for ploting the tree viz

In [ ]:
estimatorsdf.to_csv(r'C:\Users\kevinm\Documents\GitHub\7333QTW\Case Study 3\estimators.csv')


### Q1 (revised model) BUILD/EVALUTE TREE-BASED MODEL TO PREDICT "SPAM"
The grid search finds a model with very good accuracy of close to 98%. However, the model is very complex and has a depth of 120 branches. An evaluation of all the models evaluated shows that tree with a much smaller branch depth can produce a fairly accurate tree, with few branches.  This simpler tree can be used to describe some of the more important features of the data set in a more compact manner. 

In [ ]:
clf=DecisionTreeClassifier(**{'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 5})
#clf.fit(X1_train,y1_train)
#yhat = clf.predict(X1_test)
#print ('accuracy:', mt.accuracy_score(y1_test,yhat))

This will do a cv split of the data for testing and training 10 times and refit and evaluate the model. 

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
for fold, (train, test) in enumerate(cv.split(X,y)):
    print ('Next Evaluation:)
    # train the decision tree algorithm
    clf.fit(X.iloc[train],y[train])
    yhat = clf.predict(X.iloc[test])
    print ('accuracy:', mt.accuracy_score(y[test],yhat),'\n')
    conf = mt.confusion_matrix(y[test],yhat)
    print("confusion matrix\n",conf,'\n')
    print("Precision Score is: {}" .format(precision_score(y[test],yhat, average='weighted')),'\n')
    print("Recall Score is: {}" .format(recall_score(y[test],yhat, average='weighted')),'\n')
    print("F1 Score is: {}" .format(f1_score(y[test],yhat, average='weighted')),'\n')

### Q4: WHICH VARIABLES "MOST" IMPORTANT?

The features that are being considered in our model are:
* perCaps
* perHTML
* bodyCharCt
* forwards
* numDlr
* isInReplyTo
* avgWordLen
* isWrote


Merging field names and feature importances so they can be displayed together. 

In [ ]:
fi = pd.DataFrame(clf.feature_importances_, columns =['featimp'])
featuresnames = pd.DataFrame(X1_test.columns.values.tolist(), columns =['fields'])

In [ ]:
featimpdf = pd.merge(featuresnames, fi, left_index=True, right_index=True)
featimpdf = featimpdf.sort_values(by='featimp', ascending=False)
featimpdf

In [ ]:
import seaborn as sns
plt.figure(figsize=(15,8))
ax = sns.barplot(x=featimpdf.fields, y=featimpdf.featimp)
ax.set_title('Feature Importance')
plt.xlabel("Feature")
plt.ylabel("Importance")
ax.set_xticklabels(ax.get_xticklabels(), rotation=65, horizontalalignment='right')

### Q2: PLOT/ANALYZE PATHS THROUGH TREE

Explain a path or two
what did we learn?
what do the splits tell us? 

In [ ]:
from sklearn import tree
tree.plot_tree(clf);


In [ ]:
cn = ['Spam', 'NotSpam']
fig, axes = plt.subplots(nrows = 1, ncols = 1, figsize = (3,3), dpi = 300)

tree.plot_tree(clf,
               feature_names = features, 
               class_names=cn,
               filled = True);
# fig.savefig(r'C:\Users\jjsch\Downloads\Week_5_Materials_2\plottreefncn.png')

This section of code will export the decsion tree first to a dot file, then reimport and export to PDF for easier zooming and viewing of the decsion tree. 

In [ ]:
from graphviz import Digraph

In [ ]:
#export to dot file
tree.export_graphviz(clf,
                     out_file=(r"C:\Users\kevinm\Documents\GitHub\7333QTW\Case Study 3\tree.dot"),
                               #(r"C:\Users\jjsch\Downloads\Week_5_Materials_2\tree.dot"),
                     feature_names = features, 
                     class_names=cn,
                     filled = True)

In [ ]:
import pydot
from io import StringIO
from graphviz import Source
import graphviz

In [ ]:
dotfile = StringIO()

In [ ]:
#create pdf file and open
dot_data = tree.export_graphviz(clf, out_file=None, feature_names=features)
graph = graphviz.Source(dot_data) 
graph.render("name of file",view = True)

### Q5: EVALUATE PERFORMANCE OF MODEL

Confusion Matrix to show classification 

In [ ]:
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y1_test,yhat)
print(cf_matrix)

In [ ]:
import numpy as np
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, auc, roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ["{0:0.0f}".format(value) for value in
                cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')

In [ ]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

#https://graphviz.gitlab.io/_pages/Download/Download_windows.html

ylist = Y.values.astype('int64')
ylist
from sklearn.preprocessing import label_binarize
ybinary = label_binarize(ylist, classes=[0, 1])
n_classes = ybinary.shape[1]
X_train2, X_test2, y_train2, y_test2 = train_test_split(X,ybinary, test_size=0.2)
y_score = cross_val_predict(clf, X, ybinary, cv=10 ,method='predict')
from sklearn.preprocessing import label_binarize
ybinary = label_binarize(ylist, classes=[0, 1])
n_classes = ybinary.shape[1]
X_train2, X_test2, y_train2, y_test2 = train_test_split(X,ybinary, test_size=0.2)
y_score = cross_val_predict(clf, X, ybinary, cv=10 ,method='predict')

In [ ]:
#X1_train, X1_test, y1_train, y1_test
#y_score = classifier.fit(X_train3, y_train3).decision_function(X_test3)
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):

    #y1_test,yhat    
    fpr[i], tpr[i], _ = roc_curve(ybinary, y_score)
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(ybinary.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

#Plot of a ROC curve for a specific class
plt.figure()
lw = 2
plt.plot(fpr[i], tpr[i], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[0])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Spam')
plt.legend(loc="lower right")
plt.show()

### SUMMARY/CONCLUSION/FINAL THOUGHTS

An ROC curve, also known as a Reciever Operation Characteristic Curve is a plot that calculates and can be used to compare the classifiers we will used in our modeling along with the true positive rate (TPR) and false positive rate (FPR) they convey. From a topline level, you can use the Area Under the Curve (AUC) to help detirmine which can better pair an observation with the correct spam classification. An AUC score of 1.0 denotes a perfect classifier and an area of 0.5 represents a model is no better than a random guess.  

In addition to the ROC-AUC value we will use to evaluate classifiers, we will be using other evaluators as well. 
Accuracy - Accuracy is the total number of correct predictions over the total number of predictions made. Accuracy will be plotted in our AUC curve and, while not perfect, is a good singlular measure to evaluate a model. Just be careful that we aren't overclassifying unbalalnced variables.   

**Accuracy** = (TP + TN)/(TP + FP + FN + TN) 

**Precision** - Precision is the Proportions of true positives over the total number of positive outcomes whether accurately predicted or inaccurately predicted. Precision is useful to us because it will help us better understand which model is accurately picking correct classes and not resting on selecting incorrect classes.   
Precision = (TP) / (TP + FP)  

**Recall** - Also known as sensitivity, recall is the proportion of positive outcomes that were correctly classified by our model. Essentially it tells us how many values we incorrectly predicted while the precision can tell us more about what we correctly classified. Recall is good as a pair with precision, as it will help us tell if our model is overfit or selecting a single class and not training itself.   
Recall/Sensitivity = (TP) / (TP + FN)  

**F1 Score** - Another measure of accuracy that accounts for the true negatives and false positives. 

F1 score = 2(True Positive Rate * True Negatives)/(True Positives + True Negatives)  

The aim of our modeling is to focus on maximizing our precision, recall, and accuracy scores in our models  

Reference - https://github.com/jjschueder/7331DataMiningNotebooks/blob/master/lab2/Lab2_Daniel_Jeff_Armando_Joe(Final).ipynb

### Backup evaluation of complex model
The evaluation below shows cross validation of the deeper tree model with more branches.  Accuracy and other metrics are better.

In [ ]:
dt_clf=DecisionTreeClassifier(**dt_clf.best_params_)
dt_clf.fit(X1_train,y1_train)
yhat = dt_clf.predict(X1_test)
print ('accuracy:', mt.accuracy_score(y1_test,yhat))


fi_dt = pd.DataFrame(dt_clf.feature_importances_, columns =['featimp'])
featuresnames_dt = pd.DataFrame(X1_test.columns.values.tolist(), columns =['fields'])
featimpdt_df = pd.merge(featuresnames_dt, fi_dt, left_index=True, right_index=True)
featimpdt_df = featimpdf.sort_values(by='featimp', ascending=False)
featimpdt_df

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
for fold, (train, test) in enumerate(cv.split(X,y)):
    print ('Next Evaluation:')
    # train the decision tree algorithm
    dt_clf.fit(X.iloc[train],y[train])
    yhat = dt_clf.predict(X.iloc[test])
    print ('accuracy:', mt.accuracy_score(y[test],yhat),'\n')
    conf = mt.confusion_matrix(y[test],yhat)
    print("confusion matrix\n",conf,'\n')
    print("Precision Score is: {}" .format(precision_score(y[test],yhat, average='weighted')),'\n')
    print("Recall Score is: {}" .format(recall_score(y[test],yhat, average='weighted')),'\n')
    print("F1 Score is: {}" .format(f1_score(y[test],yhat, average='weighted')),'\n')

In [ ]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

#https://graphviz.gitlab.io/_pages/Download/Download_windows.html

ylistd = Y.values.astype('int64')
ylistd
from sklearn.preprocessing import label_binarize
ybinaryd = label_binarize(ylistd, classes=[0, 1])
n_classes = ybinaryd.shape[1]
X_train2d, X_test2d, y_train2d, y_test2d = train_test_split(X,ybinaryd, test_size=0.2)
y_scored = cross_val_predict(dt_clf, X, ybinaryd, cv=10 ,method='predict')
from sklearn.preprocessing import label_binarize
ybinaryd = label_binarize(ylistd, classes=[0, 1])
n_classes = ybinaryd.shape[1]
X_train2d, X_test2d, y_train2d, y_test2d = train_test_split(X,ybinary, test_size=0.2)
y_scored = cross_val_predict(dt_clf, X, ybinaryd, cv=10 ,method='predict')

In [ ]:
#X1_train, X1_test, y1_train, y1_test
#y_score = classifier.fit(X_train3, y_train3).decision_function(X_test3)
# Compute ROC curve and ROC area for each class
fprd = dict()
tprd = dict()
roc_aucd = dict()
for i in range(n_classes):

#y1_test,yhat    
    fprd[i], tprd[i], _ = roc_curve(ybinaryd, y_scored)
    roc_aucd[i] = auc(fprd[i], tprd[i])

# Compute micro-average ROC curve and ROC area
fprd["micro"], tprd["micro"], _ = roc_curve(ybinaryd.ravel(), y_scored.ravel())
roc_aucd["micro"] = auc(fprd["micro"], tprd["micro"])

#Plot of a ROC curve for a specific class
plt.figure()
lw = 2
plt.plot(fprd[i], tprd[i], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_aucd[0])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Spam')
plt.legend(loc="lower right")
plt.show()